In [1]:
import sys
!{sys.executable} -m pip install numpy
from LOTlib.Grammar import Grammar
import numpy as np

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support


Given examples like (2,4,6) people are asked how likely it is that a number, e.g., 8 is in that set

In [2]:
from math import log
from LOTlib.Hypotheses.LOTHypothesis import LOTHypothesis
from LOTlib.DataAndObjects import FunctionData

In [3]:
b = 0.05

In [133]:
# intializing basic hypothesis, will edit later
class MyHypothesis(LOTHypothesis):
    def __init__(self, **kwargs):
        LOTHypothesis.__init__(self, grammar=grammar, display="lambda x, y, z: %s", **kwargs)
        
    def __call__(self, *args):
        try:
            # try to do it from the superclass
            return LOTHypothesis.__call__(self, *args)
        except ZeroDivisionError:
            # and if we get an error, return nan
            return float("nan")

    def compute_single_likelihood(self, datum):
        # want to compute e^-b*(1 if datum is not in hypothesis space, and 0 if it is)
        # will try to use in, datum.ouput represents the set, i.e., hypothesis
#         print("value", self.value)
#         print("input", self(*datum.input))
        if self(*datum.input).issubset(datum.output):
            return log(1)
        else:
            return log(np.exp(-b))
        
#     def compile_function(self, datum):
#         print(self(*datum.input))
        
    def display_hypothesis(self):
        return self.value

I have to sample hypotheses that contain the target value in the set they produce in order to compute the probability that the target belongs to the hypotheses... 

want these production rules:

Set → AndSet and AndSet

Set → OrSet or OrSet

Set → Multiples of n

Set → Numbers between n and m

Set → Numbers containing the digit n Set → Prime numbers

Set → Powers of n Set → n

In [164]:
# adding primitives
from LOTlib.Eval import register_primitive

def union(A,B):
    return A.union(B)

def intersection(A,B):
    return A.intersection(B)

def multiples(n):
    s = set()
    i = 1
    if n == 0:
        return(s)
    while n*i < 101:
        s.add(n*i)
        i += 1
    return s

def numbersBetween(n,m):
    s= set()
    if n==m:
        s.add(n)
        return s
    if n < m:
        for i in range(n,m):
            s.add(i)
    else:
        for i in range(m,n):
            s.add(i)
    return s
        
def numbersContaining(n):
    s =set()
    for i in range(1,101):
        if str(n) in str(i):
            s.add(i)
    return s

def primes_(n):
    s = set()
    p_ = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, \
              53, 59, 61, 67, 71, 73, 79, 83, 89, 97]
    for i in range(5):
        s.add(p_[i])
    return s

def powers(n):
    s = set()
    i = 1
    if n == 1:
        s.add(1)
        return s
    elif n==0:
        s.add(0)
        return s
    while n**i < 101:
        s.add(n**i)
        i += 1
    return s

def set_n(n):
    s = set()
    s.add(n)
    return s

register_primitive(union,'union')
register_primitive(intersection,'intersection')
register_primitive(multiples,'multiples')
register_primitive(numbersBetween, 'numbersBetween')
register_primitive(numbersContaining, 'numbersContaining')
register_primitive(primes_, 'primes_')
register_primitive(powers, 'powers')
register_primitive(set_n, 'set_n')

In [165]:
# initializing grammar
grammar = Grammar(start='SET')

# adding production rules
grammar.add_rule('SET', 'union', ['SET', 'SET'], 1.0)
grammar.add_rule('SET', 'intersection', ['SET', 'SET'], 1.0)
grammar.add_rule('SET', 'multiples', ['MUL'], 1.0)
grammar.add_rule('SET', 'numbersBetween', ['EXPR', 'EXPR'], 1.0)
grammar.add_rule('SET', 'numbersContaining', ['EXPR'], 1.0)
grammar.add_rule('SET', 'primes_', ['EXPR'], 1.0)
grammar.add_rule('SET', 'powers', ['PWR'], 1.0)
grammar.add_rule('SET', 'set_n', ['EXPR'], 1.0)

# adding numbers 1 through 100 as terminals
for n in xrange(1,101):
    grammar.add_rule('EXPR', str(n), None, 1.0)
for n in xrange(5):
    grammar.add_rule('PWR', str(n), None, 1.0)
for n in xrange(49):
    grammar.add_rule('MUL', str(n), None, 1.0)

# adding variables
grammar.add_rule('EXPR', 'x', None, 1.0)
grammar.add_rule('EXPR', 'y', None, 1.0)
grammar.add_rule('EXPR', 'z', None, 1.0)

EXPR -> z	w/ p=1.0

In [63]:
h = MyHypothesis()
print(h)
print h.compute_prior()

lambda x, y, z: numbersContaining(50)
-6.71417052991


In [64]:
output = set()
for i in range(1,101):
    if i%2 == 0:
        output.add(i)

In [65]:
print(output)

set([2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 94, 96, 98, 100])


In [22]:
# this is the part I'm not sure about -- the data is a set of numbers, but what is the output?
# it's another set, the set of rule
data = [FunctionData(input=[2,4,6],output=output), FunctionData(input=[4,6,8],output=output)]
print(data)

[<2,4,6 -> set([2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 94, 96, 98, 100])>, <4,6,8 -> set([2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 94, 96, 98, 100])>]


In [122]:
from LOTlib.Inference.Samplers.MetropolisHastings import MHSampler
from LOTlib.TopN import TopN
    
tn = TopN(N=10) # store the top N

h0 = MyHypothesis()
print("h0", h0)

for h in MHSampler(h0, data, steps=10000):
    tn.add(h)

for h in tn.get_all(sorted=True):
    print h.posterior_score, h

('h0', lambda x, y, z: intersection(multiples(8), union(set_n(76), powers(4))))
('value', intersection(multiples(8), union(set_n(76), powers(4))))
('input', set([16, 64]))
('value', intersection(multiples(8), union(set_n(76), powers(4))))
('input', set([16, 64]))
('value', intersection(multiples(8), union(set_n(76), union(set_n(62), numbersContaining(58)))))
('input', set([]))
('value', intersection(multiples(8), union(set_n(76), union(set_n(62), numbersContaining(58)))))
('input', set([]))
('value', intersection(multiples(8), intersection(set_n(77), union(intersection(union(multiples(0), set_n(71)), union(set_n(87), powers(0))), numbersContaining(79)))))
('input', set([]))
('value', intersection(multiples(8), intersection(set_n(77), union(intersection(union(multiples(0), set_n(71)), union(set_n(87), powers(0))), numbersContaining(79)))))
('input', set([]))
('value', intersection(multiples(4), intersection(set_n(77), union(intersection(union(multiples(0), set_n(71)), union(set_n(87), p

('value', powers(0))
('input', set([0]))
('value', powers(0))
('input', set([0]))
('value', intersection(primes(96), numbersContaining(11)))
('input', set([11]))
('value', intersection(primes(96), numbersContaining(11)))
('input', set([11]))
('value', intersection(set_n(64), union(numbersBetween(31, 43), powers(1))))
('input', set([]))
('value', intersection(set_n(64), union(numbersBetween(31, 43), powers(1))))
('input', set([]))
('value', powers(3))
('input', set([27, 9, 3, 81]))
('value', powers(3))
('input', set([27, 9, 3, 81]))
('value', powers(2))
('input', set([64, 32, 2, 4, 8, 16]))
('value', powers(2))
('input', set([64, 32, 2, 4, 8, 16]))
('value', set_n(10))
('input', set([10]))
('value', set_n(10))
('input', set([10]))
('value', numbersContaining(76))
('input', set([76]))
('value', numbersContaining(76))
('input', set([76]))
('value', multiples(34))
('input', set([34, 68]))
('value', multiples(34))
('input', set([34, 68]))
('value', powers(3))
('input', set([27, 9, 3, 81]))


('input', set([1, 2, 3, 4, 5, 7, 11]))
('value', union(numbersBetween(34, 81), powers(1)))
('input', set([1, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80]))
('value', union(numbersBetween(34, 81), powers(1)))
('input', set([1, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80]))
('value', union(numbersBetween(34, 81), powers(1)))
('input', set([1, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80]))
('value', union(numbersBetween(34, 81), powers(1)))
('input', set([1, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61,

('value', union(primes(14), intersection(powers(4), numbersContaining(86))))
('input', set([3, 2, 11, 5, 7]))
('value', union(primes(14), intersection(powers(4), numbersContaining(86))))
('input', set([3, 2, 11, 5, 7]))
('value', union(primes(14), intersection(multiples(29), numbersContaining(86))))
('input', set([3, 2, 11, 5, 7]))
('value', union(primes(14), intersection(multiples(29), numbersContaining(86))))
('input', set([3, 2, 11, 5, 7]))
('value', union(numbersContaining(47), intersection(multiples(29), numbersContaining(86))))
('input', set([47]))
('value', union(numbersContaining(47), intersection(multiples(29), numbersContaining(86))))
('input', set([47]))
('value', union(numbersContaining(47), multiples(38)))
('input', set([76, 38, 47]))
('value', union(numbersContaining(47), multiples(38)))
('input', set([76, 38, 47]))
('value', multiples(26))
('input', set([26, 52, 78]))
('value', multiples(26))
('input', set([26, 52, 78]))
('value', primes(79))
('input', set([11, 2, 3, 5, 

('value', union(numbersContaining(29), intersection(powers(4), set_n(23))))
('input', set([29]))
('value', union(numbersContaining(29), intersection(powers(4), set_n(23))))
('input', set([29]))
('value', union(numbersContaining(29), intersection(powers(4), set_n(18))))
('input', set([29]))
('value', union(numbersContaining(29), intersection(powers(4), set_n(18))))
('input', set([29]))
('value', union(numbersContaining(29), intersection(powers(4), set_n(33))))
('input', set([29]))
('value', union(numbersContaining(29), intersection(powers(4), set_n(33))))
('input', set([29]))
('value', union(numbersContaining(29), intersection(powers(1), set_n(33))))
('input', set([29]))
('value', union(numbersContaining(29), intersection(powers(1), set_n(33))))
('input', set([29]))
('value', union(numbersContaining(29), intersection(union(union(intersection(numbersContaining(76), numbersContaining(46)), union(primes(6), numbersBetween(37, 48))), intersection(multiples(5), powers(3))), set_n(33))))
('in

...

('value', intersection(set_n(35), numbersBetween(z, 42)))
('input', set([35]))
('value', intersection(set_n(35), numbersBetween(z, 42)))
('input', set([35]))
('value', intersection(set_n(35), powers(4)))
('input', set([]))
('value', intersection(set_n(35), powers(4)))
('input', set([]))
('value', intersection(numbersBetween(33, 97), powers(4)))
('input', set([64]))
('value', intersection(numbersBetween(33, 97), powers(4)))
('input', set([64]))
('value', intersection(numbersBetween(47, 97), powers(4)))
('input', set([64]))
('value', intersection(numbersBetween(47, 97), powers(4)))
('input', set([64]))
('value', numbersContaining(39))
('input', set([39]))
('value', numbersContaining(39))
('input', set([39]))
('value', numbersContaining(92))
('input', set([92]))
('value', numbersContaining(92))
('input', set([92]))
('value', numbersContaining(23))
('input', set([23]))
('value', numbersContaining(23))
('input', set([23]))
('value', numbersContaining(56))
('input', set([56]))
('value', numb

('value', union(intersection(multiples(15), intersection(multiples(42), set_n(100))), set_n(81)))
('input', set([81]))
('value', union(intersection(multiples(15), intersection(multiples(42), set_n(100))), set_n(81)))
('input', set([81]))
('value', numbersContaining(89))
('input', set([89]))
('value', numbersContaining(89))
('input', set([89]))
('value', numbersContaining(55))
('input', set([55]))
('value', numbersContaining(55))
('input', set([55]))
('value', union(intersection(numbersContaining(42), numbersBetween(61, 77)), primes(38)))
('input', set([3, 2, 11, 5, 7]))
('value', union(intersection(numbersContaining(42), numbersBetween(61, 77)), primes(38)))
('input', set([3, 2, 11, 5, 7]))
('value', numbersContaining(77))
('input', set([77]))
('value', numbersContaining(77))
('input', set([77]))
('value', numbersContaining(90))
('input', set([90]))
('value', numbersContaining(90))
('input', set([90]))
('value', numbersContaining(24))
('input', set([24]))
('value', numbersContaining(24

('value', numbersBetween(21, 82))
('input', set([21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81]))
('value', numbersBetween(21, 82))
('input', set([21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81]))
('value', numbersBetween(21, 100))
('input', set([21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]))
('value', numbersBetween(21, 100))
('input

('value', powers(3))
('input', set([27, 9, 3, 81]))
('value', powers(3))
('input', set([27, 9, 3, 81]))
('value', powers(0))
('input', set([0]))
('value', powers(0))
('input', set([0]))
('value', powers(2))
('input', set([64, 32, 2, 4, 8, 16]))
('value', powers(2))
('input', set([64, 32, 2, 4, 8, 16]))
('value', powers(4))
('input', set([16, 64, 4]))
('value', powers(4))
('input', set([16, 64, 4]))
('value', powers(0))
('input', set([0]))
('value', powers(0))
('input', set([0]))
('value', primes(87))
('input', set([11, 2, 3, 5, 7]))
('value', primes(87))
('input', set([11, 2, 3, 5, 7]))
('value', intersection(set_n(50), set_n(6)))
('input', set([]))
('value', intersection(set_n(50), set_n(6)))
('input', set([]))
('value', powers(1))
('input', set([1]))
('value', powers(1))
('input', set([1]))
('value', multiples(38))
('input', set([76, 38]))
('value', multiples(38))
('input', set([76, 38]))
('value', multiples(13))
('input', set([65, 39, 13, 78, 52, 26, 91]))
('value', multiples(13))
(

('value', primes(4))
('input', set([11, 2, 3, 5, 7]))
('value', multiples(42))
('input', set([42, 84]))
('value', multiples(42))
('input', set([42, 84]))
('value', multiples(18))
('input', set([72, 18, 36, 90, 54]))
('value', multiples(18))
('input', set([72, 18, 36, 90, 54]))
('value', multiples(7))
('input', set([98, 35, 70, 7, 42, 77, 14, 49, 84, 21, 56, 91, 28, 63]))
('value', multiples(7))
('input', set([98, 35, 70, 7, 42, 77, 14, 49, 84, 21, 56, 91, 28, 63]))
('value', primes(34))
('input', set([11, 2, 3, 5, 7]))
('value', primes(34))
('input', set([11, 2, 3, 5, 7]))
('value', primes(99))
('input', set([11, 2, 3, 5, 7]))
('value', primes(99))
('input', set([11, 2, 3, 5, 7]))
('value', numbersContaining(81))
('input', set([81]))
('value', numbersContaining(81))
('input', set([81]))
('value', primes(48))
('input', set([11, 2, 3, 5, 7]))
('value', primes(48))
('input', set([11, 2, 3, 5, 7]))
-5.97126183979 lambda x, y, z: multiples(18)
-5.97126183979 lambda x, y, z: multiples(0)
-5.

In [184]:
def i_o(h):
    h_ = eval(str(h))
    return h.value, h_(0,0,0)

In [185]:
input_outputs = []
for i in range(100):
    h0 = MyHypothesis()
    input_outputs.append(i_o(h0))

In [188]:
print(input_outputs)[:5]

[(numbersBetween(42, 56), set([42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55])), (set_n(59), set([59])), (primes_(40), set([11, 2, 3, 5, 7])), (primes_(71), set([11, 2, 3, 5, 7])), (primes_(57), set([11, 2, 3, 5, 7]))]
